In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument('--headless')

driver = webdriver.Chrome(options=options)

In [2]:
import time
import pandas as pd
from io import StringIO

def scrape_url(year, week):
    driver.get(url=f"https://www.fantasypros.com/nfl/rankings/half-point-ppr-superflex.php?year={year}&week={week}")
    time.sleep(5)  # give time for page to load

    data = pd.read_html(StringIO(driver.page_source))[0]
    data = data[['RK', 'Player Name', 'POS']]  # opponent and matchup data is innacurate

    # get rid of team from player name
    data['Player Name'] = data['Player Name'].str.split(' \(').str[0]

    # extract ranking within position
    data['POS RK'] = data['POS'].str.extract('(\d+)').astype(int)
    data['POS'] = data['POS'].str.replace('\d+', '', regex=True)

    data.insert(0, "Year", year)
    data.insert(1, "Week", week)

    return data

# scrape_url(2023, 1)

,Year,Week,RK,Player Name,POS,POS RK
0,2023,1,1,Patrick Mahomes II,QB,1
1,2023,1,2,Jalen Hurts,QB,2
2,2023,1,3,Lamar Jackson,QB,3
3,2023,1,4,Josh Allen,QB,4
4,2023,1,5,Justin Herbert,QB,5
...,...,...,...,...,...,...
600,2023,1,601,Terrell Bynum,WR,239
601,2023,1,602,D.J. Montgomery,WR,240
602,2023,1,603,Austin Trammell,WR,241
603,2023,1,604,Joseph Ngata,WR,242


In [3]:
from itertools import product
from tqdm.autonotebook import tqdm

d = []
for year, week in tqdm(list(product([year for year in range(2018, 2024)], [week for week in range(1, 18)]))):
    if (week != 17) or (year >= 2021):
        d.append(scrape_url(year, week))

C:\Users\mhugh\AppData\Local\Temp\ipykernel_928\1166229690.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/102 [00:00<?, ?it/s]

In [4]:
df = pd.concat(d, ignore_index=True)

df

,Year,Week,RK,Player Name,POS,POS RK
0,2018,1,1,Todd Gurley II,RB,1
1,2018,1,2,David Johnson,RB,2
2,2018,1,3,Alvin Kamara,RB,3
3,2018,1,4,Melvin Gordon III,RB,4
4,2018,1,5,Ezekiel Elliott,RB,5
...,...,...,...,...,...,...
41057,2023,17,461,Steven Sims Jr.,WR,183
41058,2023,17,462,Jashaun Corbin,RB,112
41059,2023,17,463,Dee Eskridge,WR,184
41060,2023,17,464,Clayton Tune,QB,65


In [5]:
df.groupby(['Year', 'Week'])['Player Name'].count().min()

np.int64(261)

In [6]:
df.to_parquet(r'..\..\data\fantasypros_in_season_rankings\superflex_rankings.parquet')

'Done'

'Done'